<table style="width:100%; background-color: #D9EDF7">
  <tr>
    <td style="border: 1px solid #CFCFCF">
      <b>Weather data: Main notebook</b>
      <ul>
        <li><a href="main.ipynb">Main Notebook</a></li>
        <li>Downloading Notebook</li>
        <li><a href="documentation.ipynb">Documentation</a></li>
      </ul>
      <br>This Notebook is part of the <a href="http://data.open-power-system-data.org/weather_data">Weather data Datapackage</a> of <a href="http://open-power-system-data.org">Open Power System Data</a>.
    </td>
  </tr>
</table>

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Introductory-Notes" data-toc-modified-id="Introductory-Notes-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introductory Notes</a></div><div class="lev2 toc-item"><a href="#How-to-use-the-script:" data-toc-modified-id="How-to-use-the-script:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>How to use the script:</a></div><div class="lev1 toc-item"><a href="#Script-Setup" data-toc-modified-id="Script-Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Script Setup</a></div><div class="lev1 toc-item"><a href="#Download-raw-data" data-toc-modified-id="Download-raw-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Download raw data</a></div><div class="lev2 toc-item"><a href="#Input" data-toc-modified-id="Input-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Input</a></div><div class="lev3 toc-item"><a href="#Timeframe" data-toc-modified-id="Timeframe-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Timeframe</a></div><div class="lev3 toc-item"><a href="#Geography-coordinates" data-toc-modified-id="Geography-coordinates-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Geography coordinates</a></div><div class="lev2 toc-item"><a href="#Subsetting-data" data-toc-modified-id="Subsetting-data-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Subsetting data</a></div><div class="lev1 toc-item"><a href="#Downloading-data" data-toc-modified-id="Downloading-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Downloading data</a></div><div class="lev2 toc-item"><a href="#Get-wind-data" data-toc-modified-id="Get-wind-data-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Get wind data</a></div><div class="lev2 toc-item"><a href="#Get-roughness-data" data-toc-modified-id="Get-roughness-data-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Get roughness data</a></div><div class="lev2 toc-item"><a href="#Get-radiation-data" data-toc-modified-id="Get-radiation-data-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Get radiation data</a></div><div class="lev2 toc-item"><a href="#Get-temperature-data" data-toc-modified-id="Get-temperature-data-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Get temperature data</a></div><div class="lev2 toc-item"><a href="#Get-air-density-data" data-toc-modified-id="Get-air-density-data-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Get air density data</a></div><div class="lev2 toc-item"><a href="#Get-air-pressure-data" data-toc-modified-id="Get-air-pressure-data-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Get air pressure data</a></div><div class="lev2 toc-item"><a href="#Get-lat-and-lon-dimensions" data-toc-modified-id="Get-lat-and-lon-dimensions-4.7"><span class="toc-item-num">4.7&nbsp;&nbsp;</span>Get lat and lon dimensions</a></div><div class="lev2 toc-item"><a href="#Check-the-precision-of-the-downloaded-data" data-toc-modified-id="Check-the-precision-of-the-downloaded-data-4.8"><span class="toc-item-num">4.8&nbsp;&nbsp;</span>Check the precision of the downloaded data</a></div><div class="lev1 toc-item"><a href="#Setting-up-the-DataFrame" data-toc-modified-id="Setting-up-the-DataFrame-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Setting up the DataFrame</a></div><div class="lev2 toc-item"><a href="#Converting-the-timeformat-to-ISO-8601" data-toc-modified-id="Converting-the-timeformat-to-ISO-8601-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Converting the timeformat to ISO 8601</a></div><div class="lev2 toc-item"><a href="#Converting-wind-vectors-to-wind-speed" data-toc-modified-id="Converting-wind-vectors-to-wind-speed-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Converting wind vectors to wind speed</a></div><div class="lev2 toc-item"><a href="#Setting-up-data-Frame-for-roughness,-radiation,-temperature-and-air-parameters" data-toc-modified-id="Setting-up-data-Frame-for-roughness,-radiation,-temperature-and-air-parameters-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Setting up data Frame for roughness, radiation, temperature and air parameters</a></div><div class="lev2 toc-item"><a href="#Combining-the-data-Frames" data-toc-modified-id="Combining-the-data-Frames-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Combining the data Frames</a></div><div class="lev1 toc-item"><a href="#Structure-the-dataframe,-add-and-remove-columns" data-toc-modified-id="Structure-the-dataframe,-add-and-remove-columns-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Structure the dataframe, add and remove columns</a></div><div class="lev2 toc-item"><a href="#Calculating-the-displacement-height" data-toc-modified-id="Calculating-the-displacement-height-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Calculating the displacement height</a></div><div class="lev2 toc-item"><a href="#Adding-needed-and-removing-not-needed-columns" data-toc-modified-id="Adding-needed-and-removing-not-needed-columns-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Adding needed and removing not needed columns</a></div><div class="lev2 toc-item"><a href="#Renaming-and-sorting-columns" data-toc-modified-id="Renaming-and-sorting-columns-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Renaming and sorting columns</a></div><div class="lev2 toc-item"><a href="#First-look-at-the-final-data-Frame" data-toc-modified-id="First-look-at-the-final-data-Frame-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>First look at the final data Frame</a></div><div class="lev3 toc-item"><a href="#structure-and-format" data-toc-modified-id="structure-and-format-6.4.1"><span class="toc-item-num">6.4.1&nbsp;&nbsp;</span>structure and format</a></div><div class="lev3 toc-item"><a href="#resulting-dataframe" data-toc-modified-id="resulting-dataframe-6.4.2"><span class="toc-item-num">6.4.2&nbsp;&nbsp;</span>resulting dataframe</a></div><div class="lev1 toc-item"><a href="#Saving-data" data-toc-modified-id="Saving-data-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Saving data</a></div><div class="lev2 toc-item"><a href="#Saving-final-data-Frame" data-toc-modified-id="Saving-final-data-Frame-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Saving final data Frame</a></div><div class="lev2 toc-item"><a href="#Create-metadata" data-toc-modified-id="Create-metadata-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Create metadata</a></div><div class="lev2 toc-item"><a href="#Generating-checksums" data-toc-modified-id="Generating-checksums-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Generating checksums</a></div>

---


# Introductory Notes

This script contains code that allows the download, subset and processing of [MERRA-2](http://gmao.gsfc.nasa.gov/reanalysis/MERRA-2/) datasets (provided by NASA Goddard Space Flight Center) and export them as CSV.

**Weather data differ significantly from the other data types used resp. provided by OPSD** in that the sheer size of the data packages greatly exceeds OPSD's capacity to host them in a similar way as feed-in timeseries, power plant data etc. While the other data packages also offer a complete one-klick download of the bundled data packages with all relevant data this is impossible for weather datasets like MERRA-2 due to their size (variety of variables, very long timespan, huge geographical coverage etc.). It would make no sense to mirror the data from the NASA servers.

Instead we choose to provide only a **documented methodological script**. The  method describes one way to automatically obtain the desired weather data from the MERRA-2 database and simplifies resp. unifies alternative manual data obtaining methods in a single script.

**More detailed background information** on weather data can be found in the <a href="main.ipynb">Main notebook</a> and the [OPSD Wiki](https://github.com/Open-Power-System-Data/common/wiki/Information-on-weather-data) on Github.

## How to use the script:

To download MERRA-2 data, you have to **register at NASA earth data portal:**
1. Register an account at [https://urs.earthdata.nasa.gov/](https://urs.earthdata.nasa.gov/)
2. Go to "My Applications" -> "Approve More Applications" and add _NASA GESDISC DATA ARCHIVE_ (scroll down list)
3. Input your username and password when requested by the script

_Hint: It seems that the routine sometimes has problems with usernames which include upper case letters - avoid them if you can._

---
# Script Setup

In [1]:
import pandas as pd
import xarray as xr
import numpy as np
import requests
import logging
import yaml
import json
import os
import hashlib

from datetime import datetime
from calendar import monthrange
from opendap_download.multi_processing_download import DownloadManager
import math
from functools import partial
import re
import getpass
from datetime import datetime, timedelta
import dateutil.parser

# Set up a log
logging.basicConfig(level=logging.INFO)
log = logging.getLogger('notebook')

# Create output folder if it doesn't exist
os.makedirs('output', exist_ok=True)
os.makedirs('output/weather_data', exist_ok=True)

---
# Download raw data

## Input

This part defines the input parameters according to the user and creates an URL that can download the desired MERRA-2 data via the OPeNDAP interface (see <a href="documentation.ipynb">documentation notebook</a> for information on OPeNDAP).

### Timeframe

Definition of desired timespan the data is needed for. (only complete years)

In [2]:
# User input of timespan
download_year = 2016

# Create the start date 2014-01-01
download_start_date = str(download_year) + '-01-01'

### Geography coordinates

Definition of desired coordinates. The user has to input two corner coordinates 
of a rectangular area (Format WGS84, decimal system).
* Northeast coordinate: lat_1, lon_1
* Southwest coordinate: lat_2, lon_2

The area/coordinates will be converted from lat/lon to the MERRA-2 grid coordinates.
Since the resolution of the MERRA-2 grid is 0.5 x 0.625°, the given exact coordinates will 
matched as close as possible.

In [3]:
# User input of coordinates
# ------
# Example: Germany (lat/lon)
# Northeastern point: 55.05917°N, 15.04361°E
# Southwestern point: 47.27083°N, 5.86694°E

# It is important to make the southwestern coordinate lat_1 and lon_1 since
# the MERRA-2 portal requires it!
# Southwestern coordinate
# lat_1, lon_1 = 47.27083, 5.86694 Germany
# Northeastern coordinate 
# lat_2, lon_2 = 55.05917, 15.04361 Germany

# Southwestern coordinate
lat_1, lon_1 = 47.27083, 5.86694
# Northeastern coordinate
lat_2, lon_2 = 55.05917, 15.04361

def translate_lat_to_geos5_native(latitude):
    """
    The source for this formula is in the MERRA2 
    Variable Details - File specifications for GEOS pdf file.
    The Grid in the documentation has points from 1 to 361 and 1 to 576.
    The MERRA-2 Portal uses 0 to 360 and 0 to 575.
    latitude: float Needs +/- instead of N/S
    """
    return ((latitude + 90) / 0.5)

def translate_lon_to_geos5_native(longitude):
    """See function above"""
    return ((longitude + 180) / 0.625)

def find_closest_coordinate(calc_coord, coord_array):
    """
    Since the resolution of the grid is 0.5 x 0.625, the 'real world'
    coordinates will not be matched 100% correctly. This function matches 
    the coordinates as close as possible. 
    """
    # np.argmin() finds the smallest value in an array and returns its
    # index. np.abs() returns the absolute value of each item of an array.
    # To summarize, the function finds the difference closest to 0 and returns 
    # its index. 
    index = np.abs(coord_array-calc_coord).argmin()
    return coord_array[index]

# The arrays contain the coordinates of the grid used by the API.
# The values are from 0 to 360 and 0 to 575
lat_coords = np.arange(0, 361, dtype=int)
lon_coords = np.arange(0, 576, dtype=int)

# Translate the coordinates that define your area to grid coordinates.
lat_coord_1 = translate_lat_to_geos5_native(lat_1)
lon_coord_1 = translate_lon_to_geos5_native(lon_1)
lat_coord_2 = translate_lat_to_geos5_native(lat_2)
lon_coord_2 = translate_lon_to_geos5_native(lon_2)


# Find the closest coordinate in the grid.
lat_co_1_closest = find_closest_coordinate(lat_coord_1, lat_coords)
lon_co_1_closest = find_closest_coordinate(lon_coord_1, lon_coords)
lat_co_2_closest = find_closest_coordinate(lat_coord_2, lat_coords)
lon_co_2_closest = find_closest_coordinate(lon_coord_2, lon_coords)

# Check the precision of the grid coordinates. These coordinates are not lat/lon. 
# They are coordinates on the MERRA-2 grid. 
log.info('Calculated coordinates for point 1: ' + str((lat_coord_1, lon_coord_1)))
log.info('Closest coordinates for point 1: ' + str((lat_co_1_closest, lon_co_1_closest)))
log.info('Calculated coordinates for point 2: ' + str((lat_coord_2, lon_coord_2)))
log.info('Closest coordinates for point 2: ' + str((lat_co_2_closest, lon_co_2_closest)))

## Subsetting data

Combining parameter choices above/translation according to OPenDAP guidelines into URL-appendix

In [4]:
def translate_year_to_file_number(year):
    """
    The file names consist of a number and a meta data string. 
    The number changes over the years. 1980 until 1991 it is 100, 
    1992 until 2000 it is 200, 2001 until 2010 it is  300 
    and from 2011 until now it is 400.
    """
    file_number = ''
    
    if year >= 1980 and year < 1992:
        file_number = '100'
    elif year >= 1992 and year < 2001:
        file_number = '200'
    elif year >= 2001 and year < 2011:
        file_number = '300'
    elif year >= 2011:
        file_number = '400'
    else:
        raise Exception('The specified year is out of range.')
    
    return file_number
    


def generate_url_params(parameter, time_para, lat_para, lon_para):
    """Creates a string containing all the parameters in query form"""
    parameter = map(lambda x: x + time_para, parameter)
    parameter = map(lambda x: x + lat_para, parameter)
    parameter = map(lambda x: x + lon_para, parameter)
    
    return ','.join(parameter)
    
    

def generate_download_links(download_years, base_url, dataset_name, url_params):
    """
    Generates the links for the download. 
    download_years: The years you want to download as array. 
    dataset_name: The name of the data set. For example tavg1_2d_slv_Nx
    """
    urls = []
    for y in download_years: 
    # build the file_number
        y_str = str(y)
        file_num = translate_year_to_file_number(download_year)
        for m in range(1,13):
            # build the month string: for the month 1 - 9 it starts with a leading 0. 
            # zfill solves that problem
            m_str = str(m).zfill(2)
            # monthrange returns the first weekday and the number of days in a 
            # month. Also works for leap years.
            _, nr_of_days = monthrange(y, m)
            for d in range(1,nr_of_days+1):
                d_str = str(d).zfill(2)
                # Create the file name string
                file_name = 'MERRA2_{num}.{name}.{y}{m}{d}.nc4'.format(
                    num=file_num, name=dataset_name, 
                    y=y_str, m=m_str, d=d_str)
                # Create the query
                query = '{base}{y}/{m}/{name}.nc4?{params}'.format(
                    base=base_url, y=y_str, m=m_str, 
                    name=file_name, params=url_params)
                urls.append(query)
    return urls

requested_params = ['U2M', 'U10M', 'U50M', 'V2M', 'V10M', 'V50M', 'DISPH']
requested_time = '[0:1:23]'
# Creates a string that looks like [start:1:end]. start and end are the lat or
# lon coordinates define your area.
requested_lat = '[{lat_1}:1:{lat_2}]'.format(
                lat_1=lat_co_1_closest, lat_2=lat_co_2_closest)
requested_lon = '[{lon_1}:1:{lon_2}]'.format(
                lon_1=lon_co_1_closest, lon_2=lon_co_2_closest)



parameter = generate_url_params(requested_params, requested_time,
                                requested_lat, requested_lon)

BASE_URL = 'https://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/'
generated_URL = generate_download_links([download_year], BASE_URL, 
                                        'tavg1_2d_slv_Nx', 
                                        parameter)
            
# See what a query to the MERRA-2 portal looks like.        
log.info(generated_URL[0])

---
# Downloading data

This part subsequently downloads the subsetted raw data from the MERRA-2-datasets. 
The download process is outsourced from the notebook, because it is a standard and repetitive process. If you are interested in the the code, see the [opendap_download module](opendap_download/).

_Note: Each of the following steps to download the data will take a few minutes, depending on the size of geographical area and amount of data (the total download routine e.G. for Germany takes roughly 70 minutes)._

## Get wind data

Parameters from the dataset [tavg1_2d_slv_Nx (M2T1NXSLV)](http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/contents.html)

In [5]:
# Download data (one file per day and dataset) with links to local directory.
# Username and password for MERRA-2 (NASA earthdata portal)
username = input('Username: ')
password = getpass.getpass('Password:')

# The DownloadManager is able to download files. If you have a fast internet 
# connection, setting this to 2 is enough. If you have slow wifi, try setting
# it to 4 or 5. If you download too fast, the data portal might ban you for a 
# day. 
NUMBER_OF_CONNECTIONS = 2

# The DownloadManager class is defined in the opendap_download module. 
download_manager = DownloadManager()
download_manager.set_username_and_password(username, password)
download_manager.download_path = 'download_wind'
download_manager.download_urls = generated_URL

# If you want to see the download progress, check the download folder you 
# specified
%time download_manager.start_download(NUMBER_OF_CONNECTIONS)

# Download time approx. 15+ min.

Username: opsd_new
Password:········
Wall time: 24min 34s


## Get roughness data

Parameters from the dataset [tavg1_2d_rad_Nx (M2T1NXRAD)](https://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXRAD.5.12.4/contents.html)

In [ ]:
# Roughness data is in a different data set. The parameter is called Z0M. 
roughness_para = generate_url_params(['Z0M'], requested_time, 
                                     requested_lat, requested_lon)
ROUGHNESS_BASE_URL = 'https://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXFLX.5.12.4/'
roughness_links = generate_download_links([download_year], ROUGHNESS_BASE_URL,
                                          'tavg1_2d_flx_Nx', roughness_para)
            
download_manager.download_path = 'download_roughness'
download_manager.download_urls = roughness_links

# If you want to see the download progress, check the download folder you 
# specified.
%time download_manager.start_download(NUMBER_OF_CONNECTIONS)

# Download time approx. 12+ min.

## Get radiation data

Parameters from the dataset [tavg1_2d_flx_Nx (M2T1NXFLX)](http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXFLX.5.12.4/contents.html)

In [ ]:
# Parameters SWGDN and SWTDN
radiation_para = generate_url_params(['SWGDN', 'SWTDN'], requested_time, 
                                     requested_lat, requested_lon)
RADIATION_BASE_URL = 'https://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXRAD.5.12.4/'
radiation_links = generate_download_links([download_year], RADIATION_BASE_URL, 
                                         'tavg1_2d_rad_Nx', radiation_para)

download_manager.download_path = 'download_radiation'
download_manager.download_urls = radiation_links

%time download_manager.start_download(NUMBER_OF_CONNECTIONS)

# Download time approx. 8+ min.

## Get temperature data

Parameters from the dataset [tavg1_2d_slv_Nx (M2T1NXSLV)](http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/contents.html)

In [ ]:
# Parameter T2M (i.e. the temperature 2 meters above displacement height)
temperature_para = generate_url_params(['T2M'], requested_time, 
                                     requested_lat, requested_lon)
TEMPERATURE_BASE_URL = 'http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/'
temperature_links = generate_download_links([download_year], TEMPERATURE_BASE_URL, 
                                         'tavg1_2d_slv_Nx', temperature_para)

download_manager.download_path = 'download_temperature'
download_manager.download_urls = temperature_links

%time download_manager.start_download(NUMBER_OF_CONNECTIONS)

# Download time approx. 13+ min.

## Get air density data

Parameters from the dataset [tavg1_2d_flx_Nx (M2T1NXFLX)](http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXFLX.5.12.4/contents.html)

In [ ]:
# Parameter RHOA
density_para = generate_url_params(['RHOA'], requested_time, 
                                     requested_lat, requested_lon)
DENSITY_BASE_URL = 'http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXFLX.5.12.4/'
density_links = generate_download_links([download_year], DENSITY_BASE_URL, 
                                         'tavg1_2d_flx_Nx', density_para)

download_manager.download_path = 'download_density'
download_manager.download_urls = density_links

%time download_manager.start_download(NUMBER_OF_CONNECTIONS)

# Download time approx. 13+ min.

## Get air pressure data

Parameters from the dataset [tavg1_2d_slv_Nx (M2T1NXSLV)](http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/contents.html)

In [19]:
# Parameters PS
pressure_para = generate_url_params(['PS'], requested_time, 
                                     requested_lat, requested_lon)
PRESSURE_BASE_URL = 'http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/'
pressure_links = generate_download_links([download_year], PRESSURE_BASE_URL, 
                                         'tavg1_2d_slv_Nx', pressure_para)

download_manager.download_path = 'download_pressure'
download_manager.download_urls = pressure_links

%time download_manager.start_download(NUMBER_OF_CONNECTIONS)

# Download time approx. 15+ min.

Wall time: 22min 34s


## Get lat and lon dimensions

For now, the dataset only has MERRA-2 grid coordinates. To translate the points
back to "real world" coordinates, the data portal offers a dimension scale file.

In [7]:
# The dimensions map the MERRA2 grid coordinates to lat/lon. The coordinates 
# to request are 0:360 wheare as the other coordinates are 1:361
requested_lat_dim = '[{lat_1}:1:{lat_2}]'.format(
                    lat_1=lat_co_1_closest, lat_2=lat_co_2_closest)
requested_lon_dim = '[{lon_1}:1:{lon_2}]'.format(
                    lon_1=lon_co_1_closest , lon_2=lon_co_2_closest )

lat_lon_dimension_para = 'lat' + requested_lat_dim + ',lon' + requested_lon_dim

# Creating the download url.
dimension_url = 'https://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/2014/01/MERRA2_400.tavg1_2d_slv_Nx.20140101.nc4.nc4?'
dimension_url = dimension_url + lat_lon_dimension_para
download_manager.download_path = 'dimension_scale'
download_manager.download_urls = [dimension_url]

# Since the dimension is only one file, we only need one connection. 
%time download_manager.start_download(1)


Wall time: 6.93 s


## Check the precision of the downloaded data

Due to the back and forth conversion from "real world" coordinates to MERRA-2 grid points,
this part helps you to check if the conversion was precise enough.

In [8]:
file_path = os.path.join('dimension_scale', DownloadManager.get_filename(
        dimension_url))

with xr.open_dataset(file_path) as ds_dim:
    df_dim = ds_dim.to_dataframe()

lat_array = ds_dim['lat'].data.tolist()
lon_array = ds_dim['lon'].data.tolist()

# The log output helps evaluating the precision of the received data.
log.info('Requested lat: ' + str((lat_1, lat_2)))
log.info('Received lat: ' + str(lat_array))
log.info('Requested lon: ' + str((lon_1, lon_2)))
log.info('Received lon: ' + str(lon_array))

---
# Setting up the DataFrame

This part sets up a DataFrame and reads the raw data into it. First the wind data and adding the remaining data afterwards.

In [26]:
def extract_date(data_set):
    """
    Extracts the date from the filename before merging the datasets. 
    """
    try:
        # The attribute name changed during the development of this script
        # from HDF5_Global.Filename to Filename. 
        if 'HDF5_GLOBAL.Filename' in data_set.attrs:
            f_name = data_set.attrs['HDF5_GLOBAL.Filename']
        elif 'Filename' in data_set.attrs:
            f_name = data_set.attrs['Filename']
        else: 
            raise AttributeError('The attribute name has changed again!')
        
        # find a match between "." and ".nc4" that does not have "." .
        exp = r'(?<=\.)[^\.]*(?=\.nc4)'
        res = re.search(exp, f_name).group(0)
        # Extract the date. 
        y, m, d = res[0:4], res[4:6], res[6:8]
        date_str = ('%s-%s-%s' % (y, m, d))
        data_set = data_set.assign(date=date_str)
        return data_set

    except KeyError:
        # The last dataset is the one all the other sets will be merged into. 
        # Therefore, no date can be extracted.
        return data_set
        

file_path = os.path.join('download_wind', '*.nc4')

try:
    with xr.open_mfdataset(file_path, concat_dim='date',
                           preprocess=extract_date) as ds_wind:
        print(ds_wind)
        df_wind = ds_wind.to_dataframe()
        
except xr.MergeError as e:
    print(e)

<xarray.Dataset>
Dimensions:  (date: 366, lat: 16, lon: 16, time: 24)
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * lat      (lat) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
  * lon      (lon) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
  * date     (date) <U10 '2016-01-01' '2016-01-02' '2016-01-03' '2016-01-04' ...
Data variables:
    U10M     (date, time, lat, lon) float64 1.714 1.506 1.839 1.667 1.491 ...
    V2M      (date, time, lat, lon) float64 0.3361 0.4499 0.2888 0.08178 ...
    U50M     (date, time, lat, lon) float64 3.067 3.151 3.602 2.804 2.381 ...
    V50M     (date, time, lat, lon) float64 1.364 1.915 1.188 0.2906 -0.802 ...
    V10M     (date, time, lat, lon) float64 0.7777 0.9255 0.609 0.1743 ...
    DISPH    (date, time, lat, lon) float64 0.2666 0.8315 1.455 0.5557 ...
    U2M      (date, time, lat, lon) float64 0.7416 0.7208 0.893 0.8402 ...


In [27]:
df_wind.reset_index(inplace=True)

In [28]:
start_date = datetime.strptime(download_start_date, '%Y-%m-%d')

def calculate_datetime(d_frame):
    """
    Calculates the accumulated hour based on the date.
    """
    cur_date = datetime.strptime(d_frame['date'], '%Y-%m-%d')
    hour = int(d_frame['time'])
    delta = abs(cur_date - start_date).days
    date_time_value = (delta * 24) + (hour)
    return date_time_value


df_wind['date_time_hours'] = df_wind.apply(calculate_datetime, axis=1)
df_wind

,date,lat,lon,time,U10M,V2M,U50M,V50M,V10M,DISPH,U2M,date_time_hours
0,2016-01-01,0,0,0,1.713974,0.336058,3.066632,1.364060,0.777717,0.266602,0.741624,0
1,2016-01-01,0,0,1,1.436833,0.350907,2.368633,1.136912,0.719556,0.266602,0.689113,1
2,2016-01-01,0,0,2,0.915276,0.418872,1.404379,1.258736,0.810442,0.266479,0.511291,2
3,2016-01-01,0,0,3,0.429794,0.913696,0.549502,1.528788,1.275092,0.266479,0.304477,3
4,2016-01-01,0,0,4,-0.081605,1.140184,-0.100645,1.824697,1.562668,0.266357,-0.050926,4
5,2016-01-01,0,0,5,-0.480287,1.286391,-0.585891,2.077779,1.773434,0.266357,-0.331961,5
6,2016-01-01,0,0,6,-0.951761,1.357493,-1.154728,2.226837,1.886804,0.266357,-0.665635,6
7,2016-01-01,0,0,7,-1.502329,1.238752,-1.817245,2.039587,1.724892,0.266235,-1.056170,7
8,2016-01-01,0,0,8,-2.001586,0.875214,-2.386062,1.367085,1.175264,0.266235,-1.460677,8
9,2016-01-01,0,0,9,-2.377268,0.434705,-2.790058,0.740812,0.569060,0.266113,-1.783593,9


## Converting the timeformat to ISO 8601

In [30]:
def converting_timeformat_to_ISO8601(row):
    """Generates datetime according to ISO 8601 (UTC)"""
    date = dateutil.parser.parse(row['date'])
    hour = int(row['time'])
    # timedelta from the datetime module enables the programmer 
    # to add time to a date. 
    date_time = date + timedelta(hours = hour)
    return str(date_time.isoformat()) + 'Z'  # MERRA2 datasets have UTC time zone.
df_wind['date_utc'] = df_wind.apply(converting_timeformat_to_ISO8601, axis=1)

df_wind['date_utc']

# execution time approx. 3+ min

0          2016-01-01T00:00:00Z
1          2016-01-01T01:00:00Z
2          2016-01-01T02:00:00Z
3          2016-01-01T03:00:00Z
4          2016-01-01T04:00:00Z
5          2016-01-01T05:00:00Z
6          2016-01-01T06:00:00Z
7          2016-01-01T07:00:00Z
8          2016-01-01T08:00:00Z
9          2016-01-01T09:00:00Z
10         2016-01-01T10:00:00Z
11         2016-01-01T11:00:00Z
12         2016-01-01T12:00:00Z
13         2016-01-01T13:00:00Z
14         2016-01-01T14:00:00Z
15         2016-01-01T15:00:00Z
16         2016-01-01T16:00:00Z
17         2016-01-01T17:00:00Z
18         2016-01-01T18:00:00Z
19         2016-01-01T19:00:00Z
20         2016-01-01T20:00:00Z
21         2016-01-01T21:00:00Z
22         2016-01-01T22:00:00Z
23         2016-01-01T23:00:00Z
24         2016-01-01T00:00:00Z
25         2016-01-01T01:00:00Z
26         2016-01-01T02:00:00Z
27         2016-01-01T03:00:00Z
28         2016-01-01T04:00:00Z
29         2016-01-01T05:00:00Z
                   ...         
2248674 

## Converting wind vectors to wind speed

This part uses the given wind vectors in the MERRA-2 original data to calculate a wind speed (vector addition).

In [29]:
def calculate_windspeed(d_frame, idx_u, idx_v):
    """
    Calculates the windspeed. The returned unit is m/s
    """
    um = float(d_frame[idx_u])
    vm = float(d_frame[idx_v])
    speed = math.sqrt((um ** 2) + (vm ** 2))
    return round(speed, 2)

# partial is used to create a function with pre-set arguments. 
calc_windspeed_2m = partial(calculate_windspeed, idx_u='U2M', idx_v='V2M')
calc_windspeed_10m = partial(calculate_windspeed, idx_u='U10M', idx_v='V10M')
calc_windspeed_50m = partial(calculate_windspeed, idx_u='U50M', idx_v='V50M')

df_wind['v_2m'] = df_wind.apply(calc_windspeed_2m, axis=1)
df_wind['v_10m']= df_wind.apply(calc_windspeed_10m, axis=1)
df_wind['v_50m'] = df_wind.apply(calc_windspeed_50m, axis=1)
df_wind

# execution time approx. 3 min

,date,lat,lon,time,U10M,V2M,U50M,V50M,V10M,DISPH,U2M,date_time_hours,v_2m,v_10m,v_50m
0,2016-01-01,0,0,0,1.713974,0.336058,3.066632,1.364060,0.777717,0.266602,0.741624,0,0.81,1.88,3.36
1,2016-01-01,0,0,1,1.436833,0.350907,2.368633,1.136912,0.719556,0.266602,0.689113,1,0.77,1.61,2.63
2,2016-01-01,0,0,2,0.915276,0.418872,1.404379,1.258736,0.810442,0.266479,0.511291,2,0.66,1.22,1.89
3,2016-01-01,0,0,3,0.429794,0.913696,0.549502,1.528788,1.275092,0.266479,0.304477,3,0.96,1.35,1.62
4,2016-01-01,0,0,4,-0.081605,1.140184,-0.100645,1.824697,1.562668,0.266357,-0.050926,4,1.14,1.56,1.83
5,2016-01-01,0,0,5,-0.480287,1.286391,-0.585891,2.077779,1.773434,0.266357,-0.331961,5,1.33,1.84,2.16
6,2016-01-01,0,0,6,-0.951761,1.357493,-1.154728,2.226837,1.886804,0.266357,-0.665635,6,1.51,2.11,2.51
7,2016-01-01,0,0,7,-1.502329,1.238752,-1.817245,2.039587,1.724892,0.266235,-1.056170,7,1.63,2.29,2.73
8,2016-01-01,0,0,8,-2.001586,0.875214,-2.386062,1.367085,1.175264,0.266235,-1.460677,8,1.70,2.32,2.75
9,2016-01-01,0,0,9,-2.377268,0.434705,-2.790058,0.740812,0.569060,0.266113,-1.783593,9,1.84,2.44,2.89


## Setting up data Frame for roughness, radiation, temperature and air parameters

In [31]:
file_path = os.path.join('download_roughness', '*.nc4')
with xr.open_mfdataset(file_path, concat_dim='date', 
                       preprocess=extract_date) as ds_rough:
    df_rough = ds_rough.to_dataframe()

df_rough.reset_index(inplace=True)

In [32]:
file_path = os.path.join('download_radiation', '*.nc4')
try:
    with xr.open_mfdataset(file_path, concat_dim='date',
                           preprocess=extract_date) as ds_rad:
        print(ds_rad)
        df_rad = ds_rad.to_dataframe()

except xr.MergeError as e:
    print(e)
df_rad.reset_index(inplace=True)

<xarray.Dataset>
Dimensions:  (date: 366, lat: 16, lon: 16, time: 24)
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * lat      (lat) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
  * lon      (lon) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
  * date     (date) <U10 '2016-01-01' '2016-01-02' '2016-01-03' '2016-01-04' ...
Data variables:
    SWGDN    (date, time, lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    SWTDN    (date, time, lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...


In [33]:
file_path = os.path.join('download_temperature', '*.nc4')
try:
    with xr.open_mfdataset(file_path, concat_dim='date',
                           preprocess=extract_date) as ds_temp:
        print(ds_temp)
        df_temp = ds_temp.to_dataframe()

except xr.MergeError as e:
    print(e)
df_temp.reset_index(inplace=True)

<xarray.Dataset>
Dimensions:  (date: 366, lat: 16, lon: 16, time: 24)
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * lat      (lat) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
  * lon      (lon) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
  * date     (date) <U10 '2016-01-01' '2016-01-02' '2016-01-03' '2016-01-04' ...
Data variables:
    T2M      (date, time, lat, lon) float64 277.4 277.6 276.9 276.4 276.0 ...


In [34]:
file_path = os.path.join('download_density', '*.nc4')
try:
    with xr.open_mfdataset(file_path, concat_dim='date',
                           preprocess=extract_date) as ds_dens:
        print(ds_dens)
        df_dens = ds_dens.to_dataframe()

except xr.MergeError as e:
    print(e)
df_dens.reset_index(inplace=True)

<xarray.Dataset>
Dimensions:  (date: 366, lat: 16, lon: 16, time: 24)
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * lat      (lat) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
  * lon      (lon) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
  * date     (date) <U10 '2016-01-01' '2016-01-02' '2016-01-03' '2016-01-04' ...
Data variables:
    RHOA     (date, time, lat, lon) float64 1.236 1.226 1.205 1.2 1.202 ...


In [35]:
file_path = os.path.join('download_pressure', '*.nc4')
try:
    with xr.open_mfdataset(file_path, concat_dim='date',
                           preprocess=extract_date) as ds_pres:
        print(ds_pres)
        df_pres = ds_pres.to_dataframe()

except xr.MergeError as e:
    print(e)
df_pres.reset_index(inplace=True)

<xarray.Dataset>
Dimensions:  (date: 366, lat: 16, lon: 16, time: 24)
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * lat      (lat) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
  * lon      (lon) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
  * date     (date) <U10 '2016-01-01' '2016-01-02' '2016-01-03' '2016-01-04' ...
Data variables:
    PS       (date, time, lat, lon) float64 9.928e+04 9.819e+04 9.619e+04 ...


## Combining the data Frames

In [36]:
df = pd.merge(df_wind, df_rough, on=['date', 'lat', 'lon', 'time'])
df = pd.merge(df, df_rad, on=['date', 'lat', 'lon', 'time'])
df = pd.merge(df, df_temp, on=['date', 'lat', 'lon', 'time'])
df = pd.merge(df, df_dens, on=['date', 'lat', 'lon', 'time'])
df = pd.merge(df, df_pres, on=['date', 'lat', 'lon', 'time'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2248704 entries, 0 to 2248703
Data columns (total 22 columns):
date               object
lat                int64
lon                int64
time               int64
U10M               float64
V2M                float64
U50M               float64
V50M               float64
V10M               float64
DISPH              float64
U2M                float64
date_time_hours    int64
v_2m               float64
v_10m              float64
v_50m              float64
date_utc           object
Z0M                float64
SWGDN              float64
SWTDN              float64
T2M                float64
RHOA               float64
PS                 float64
dtypes: float64(16), int64(4), object(2)
memory usage: 394.6+ MB


---
# Structure the dataframe, add and remove columns

## Calculating the displacement height

The so-called "displacement height" is the height
> _"[...] at which zero wind speed is achieved as a result of flow obstacles such as trees or buildings. It is generally approximated as 2/3 of the average height of the obstacles. For example, if estimating winds over a forest canopy of height h = 30 m, the zero-plane displacement would be d = 20 m."_ ([Source](https://en.wikipedia.org/wiki/Log_wind_profile#Definition))

In [37]:
# Calculate height for h1 (displacement height +2m) and h2 (displacement height
# +10m).
df['h1'] = df.apply((lambda x:int(x['DISPH']) + 2), axis=1)
df['h2'] = df.apply((lambda x:int(x['DISPH']) + 10), axis=1)

## Adding needed and removing not needed columns

In [38]:
df.drop('DISPH', axis=1, inplace=True)
df.drop(['time', 'date'], axis=1, inplace=True)
df.drop(['U2M', 'U10M', 'U50M', 'V2M', 'V10M', 'V50M'], axis=1, inplace=True)

df['lat'] = df['lat'].apply(lambda x: lat_array[int(x)])
df['lon'] = df['lon'].apply(lambda x: lon_array[int(x)])

## Renaming and sorting columns

In [39]:
rename_map = {'date_time_hours': 'cumulated hours', 
              'date_utc': 'timestamp',
              'v_2m': 'v1', 
              'v_10m': 'v2', 
              'Z0M': 'z0',
              'T2M': 'T',
              'RHOA': 'rho',
              'PS': 'p'
             }

df.rename(columns=rename_map, inplace=True)

In [40]:
# Change order of the columns
columns = ['timestamp', 'cumulated hours', 'lat', 'lon',
        'v1', 'v2', 'v_50m',
        'h1', 'h2', 'z0', 'SWTDN', 'SWGDN', 'T', 'rho', 'p']
df = df[columns]

## First look at the final data Frame

### structure and format

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2248704 entries, 0 to 2248703
Data columns (total 15 columns):
timestamp          object
cumulated hours    int64
lat                float64
lon                float64
v1                 float64
v2                 float64
v_50m              float64
h1                 int64
h2                 int64
z0                 float64
SWTDN              float64
SWGDN              float64
T                  float64
rho                float64
p                  float64
dtypes: float64(11), int64(3), object(1)
memory usage: 274.5+ MB


### resulting dataframe

In [42]:
df

,timestamp,cumulated hours,lat,lon,v1,v2,v_50m,h1,h2,z0,SWTDN,SWGDN,T,rho,p
0,2016-01-01T00:00:00Z,0,47.5,5.625,0.81,1.88,3.36,2,10,0.052526,0.000000,0.000000,277.350159,1.236413,99282.710938
1,2016-01-01T01:00:00Z,1,47.5,5.625,0.77,1.61,2.63,2,10,0.052510,0.000000,0.000000,277.025665,1.239390,99300.164062
2,2016-01-01T02:00:00Z,2,47.5,5.625,0.66,1.22,1.89,2,10,0.052495,0.000000,0.000000,277.223755,1.243861,99310.992188
3,2016-01-01T03:00:00Z,3,47.5,5.625,0.96,1.35,1.62,2,10,0.052480,0.000000,0.000000,277.133240,1.247390,99314.773438
4,2016-01-01T04:00:00Z,4,47.5,5.625,1.14,1.56,1.83,2,10,0.052480,0.000000,0.000000,276.867767,1.248869,99324.796875
5,2016-01-01T05:00:00Z,5,47.5,5.625,1.33,1.84,2.16,2,10,0.052465,0.000000,0.000000,276.581299,1.250242,99324.320312
6,2016-01-01T06:00:00Z,6,47.5,5.625,1.51,2.11,2.51,2,10,0.052449,0.000000,0.000000,276.347198,1.250865,99305.062500
7,2016-01-01T07:00:00Z,7,47.5,5.625,1.63,2.29,2.73,2,10,0.052434,27.453129,4.265625,276.269531,1.250548,99271.078125
8,2016-01-01T08:00:00Z,8,47.5,5.625,1.70,2.32,2.75,2,10,0.052434,189.437500,45.843750,276.744415,1.249260,99248.015625
9,2016-01-01T09:00:00Z,9,47.5,5.625,1.84,2.44,2.89,2,10,0.052419,334.000000,146.000000,277.890259,1.244758,99227.468750


---
# Saving data

## Saving final data Frame

In [43]:
path_package = 'output'
os.makedirs(path_package, exist_ok=True)

df.to_csv(path_package + 'weather_data_result.csv', index=False)

## Create metadata

In [2]:
metadata = """
name: opsd-weather-data
title: Weather data
deslong_description: >-
    Weather data differ significantly from the other data types used resp. 
    provided by OPSD in that the sheer size of the data packages greatly 
    exceeds OPSD's capacity to host them in a similar way as feed-in 
    timeseries, power plant data etc. While the other data packages also
    offer a complete one-klick download of the bundled data packages with 
    all relevant data this is impossible for weather datasets like MERRA-2 due 
    to their size (variety of variables, very long timespan, huge geographical
    coverage etc.). It would make no sense to mirror the data from the NASA 
    servers.
    Instead we choose to provide a documented methodological script 
    (as a kind of tutorial). The method describes one way to automatically 
    obtain the desired weather data from the MERRA-2 database and simplifies 
    resp. unifies alternative manual data obtaining methods in a single 
    script.
    It is recommended to study the the "Step-by-step user guide" (developer use 
    case) on this platform to learn how to run the script.
version: "2017-06-22"
keywords: [Open Power System Data, MERRA-2, wind, solar, temperature, density, 
            pressure]
geographical-scope: Worldwide
description: Script for the download of MERRA-2 weather data


licenses:
    - type: MIT license
      url: http://www.opensource.org/licenses/MIT
sources:
    - name: MERRA-2 
      web: https://gmao.gsfc.nasa.gov/reanalysis/MERRA-2/
      source: National Aeronautics and Space Administration - Goddard Space Flight Center
contributors:
    - name: Martin Jahn
      email: martin.jahn@uni-flensburg.de
      web: 
    - name: Jan Urbansky
      email: jan.urbansky@uni-flensburg.de

views: True
documentation: https://github.com/Open-Power-System-Data/weather_data/blob/2017-06-22/main.ipynb
last_changes: added download of solar, temperature, air density and pressure data
"""

metadata = yaml.load(metadata)

datapackage_json = json.dumps(metadata, indent=4, separators=(',', ': '))

with open('datapackage.json', 'w') as f:
    f.write(datapackage_json)

## Generating checksums

In [54]:
def get_sha_hash(path, blocksize=65536):
    sha_hasher = hashlib.sha256()
    with open(path, 'rb') as f:
        buffer = f.read(blocksize)
        while len(buffer) > 0:
            sha_hasher.update(buffer)
            buffer = f.read(blocksize)
        return sha_hasher.hexdigest()


output_path = 'output'

files = ['weather_data_result.csv']

with open(os.path.join(output_path, 'checksums.txt'), 'w') as f:
    for file_name in files:
        file_hash = get_sha_hash(os.path.join(output_path, file_name))
        f.write('{},{}\n'.format(file_name, file_hash))